In [1]:
import netket as nk
import json
from qutip import *
import numpy as np
import time
import multiprocessing as mp
from collections import OrderedDict
from pickle import dump
import os
import matplotlib.pyplot as plt
import scipy
from matplotlib import gridspec
from functools import reduce
plt.style.use('seaborn')
from scipy.stats import norm
import expect_grad_ex
import vmc_ex
import jax


In [2]:
def CSHam(N, B, Ak):
    # Make graph with of length N with no periodic boundary conditions
    g = nk.graph.Hypercube(length=N, n_dim=1, pbc=False)
    # Spin based Hilbert Space
    hilbertSpace = nk.hilbert.Spin(s=0.5, N=g.n_nodes)
    # Define spin operators with \hbar set to 1
    sz = 0.5 * np.array([[1, 0], [0, -1]])
    sx = 0.5 * np.array([[0, 1], [1, 0]])
    sy = 0.5 * np.array([[0, -1j], [1j, 0]])
    operators = []
    sites = []
    # Central spin term
    operators.append((B * sz).tolist()) #array to list(ordered and changeable)
    sites.append([0])
    # Interaction term
    itOp = np.kron(sz, sz) + np.kron(sx, sx) + np.kron(sy, sy) #kronecker product here
    for i in range(N - 1):
        operators.append((Ak[i] * itOp).tolist())
        sites.append([0, (i+1)])  #pretty convoluted indexing, but ok
    # Create hamiltonian
    hamiltonian = nk.operator.LocalOperator(hilbertSpace, operators=operators, acting_on=sites, dtype=complex)
    #acting_on specifier necessary as this is a central spin model
    return hamiltonian, hilbertSpace

def exactDiagonalization(hamiltonian):
    # Changes Hamiltonian to matrix form, where hamiltonian of interest is sparse in matrix form
    #haMatrix = hamiltonian.to_sparse()
    # Gets eigenvalues and vectors, where the built-in function uses 
    eigenValues, v = nk.exact.lanczos_ed(hamiltonian, compute_eigenvectors=True)

    # Orders from smallest to largest
    eigenVectors = [v[:, i] for i in range(len(eigenValues))]
    return eigenValues, eigenVectors

def exactDiagonalization_full(hamiltonian):
    # Changes Hamiltonian to matrix form
    haMatrix = hamiltonian.to_dense()
    # Gets eigenvalues and vectors
    eigenValues, v = np.linalg.eigh(haMatrix)
    # Orders from smallest to largest
    eigenVectors = [v[:, i] for i in range(len(eigenValues))]
    return eigenValues, eigenVectors

# NetKet RBM with stochastic reconfiguration descent
class RBM:
    def __init__(self, N, hamiltonian, hilbertSpace, machine):
        # Assign inputsv[:, i]
        self.hamiltonian, self.hilbertSpace, self.machine, self.N = hamiltonian, hilbertSpace, machine, N
        # Define sampler
        self.sampler = nk.sampler.MetropolisLocal(hilbert=hilbertSpace)
        # Define optimizer
        self.optimizer = nk.optimizer.Sgd(learning_rate=0.02)
        # Define Stochastic reconfiguration
        self.sr = nk.optimizer.SR(diag_shift=0.01) #diagnol shift, its role as regularizer? seems to take a different form as 
        #compared to the version I have seen
        # Variational state
        self.vs = nk.vqs.MCState(self.sampler, self.machine, n_samples=10000, n_discard_per_chain=1000) #discarded number of samples 
        #at the beginning of the MC chain

    # Output is the name of the output file in which the descent data is stored
    def __call__(self, output, state_list, shift_list):
        self.vs.init_parameters(jax.nn.initializers.normal(stddev=0.25))
        gs = vmc_ex.VMC_ex(hamiltonian=self.hamiltonian, optimizer=self.optimizer, variational_state=self.vs, preconditioner=self.sr, 
                   state_list = state_list, shift_list = shift_list)
        # Start timing
        start = time.time()
        # Set the output files as well as number of iterations in the descent
        gs.run(out=output, n_iter=2000)
        end = time.time()
        runTime = end - start
        # Import the data from log file
        data = json.load(open(output + '.log'))
        # Extract the relevant information
        # iters = data["Energy"]["iters"]
        energy_RBM = data["Energy"]["Mean"]["real"] #get the real part of the mean energy
       
        # finalEng = energy_RBM[-1]
        finalEng = reduce(lambda x, y: x if y is None else y, energy_RBM)
        # Get machine statethe state of the machine as an array
        state = self.vs.to_array()
        # Outputs the final energy, the final state, and the runtime
        parameters = self.vs.parameters
        # Outputs the final energy, the final state, and the runtime
        return finalEng, state, self.vs

In [3]:
# Error Calculation (Input: the found state, the state from exact diagonalization, the found energy, the energy from exact diagonalization)
def err(state, edState, eng, edEng,N):
    engErr = np.abs(eng - edEng)
    overlap = np.dot(state.conj().reshape(2**N, 1).T, edState.reshape(2**N, 1))
    waveFunctionErr = 1 - (np.linalg.norm(overlap))**2
    return engErr, waveFunctionErr


# Combines all steps into a function to run on the cluster
def runDescentCS(N,B,Ak,alpha, state_list, shift_list):
    # Define hamiltonian and hibert space (need to do this here cause can't use netket objects as input to use multiprocessing functions)
    ha, hi = CSHam(N,B,Ak)
    # RBM Spin Machine
    ma = nk.models.RBM(alpha=1, dtype=complex,use_visible_bias=True, use_hidden_bias=True)
    # Initialize RBM
    rbm = RBM(N, ha, hi, ma) #an instance of class RBM
    # Run RBM
    eng, state, vstate = rbm("2021_summer_data/Logs"+str(N), state_list, shift_list) #where _call_ will be invoked
    return eng, state, vstate

In [4]:
N = 4
Ak = []

alpha = 1   #density of RBM
M = alpha*N
# Constant A
B = 0.95
# Variable A
A = N/2
N0 = N/2
for i in range(N-1):
    # Constant A
    Ak_i = 1
    # Variable A
    #Ak_i = A / (N0) * np.exp(-i / N0)
    Ak.append(Ak_i)
    
# Define hamiltonian and hilbert space
ha, hi = CSHam(N,B,Ak)


#Exact Diagonalization¢
e, v = exactDiagonalization_full(ha)

ma = nk.models.RBM(alpha=1, dtype=complex,use_visible_bias=True, use_hidden_bias=True)
sampler = nk.sampler.MetropolisLocal(hilbert=hi)
vs1 = nk.vqs.MCState(sampler, ma, n_samples=10000, n_discard_per_chain=1000)
vs1.init_parameters(jax.nn.initializers.normal(stddev=0.25))
state_list = [vs1]
shift_list = [0]

eng, state, gs = runDescentCS(N,B,Ak,alpha, state_list, shift_list)

100%|█| 2000/2000 [03:46<00:00,  8.84it/s, Energy=-1.55351+0.00001j ± 0.00040 [σ


In [43]:
print(e)

[-1.55408014 -1.35707949 -1.11638617 -0.93965571 -0.93965571 -0.225
 -0.225       0.275       0.43965571  0.43965571  0.61638617  0.725
  0.725       0.85707949  1.05408014  1.225     ]


In [6]:
state_list1 = [gs]
shift_list1 = [0.1]
eng1, state1, st1 = runDescentCS(N,B,Ak,alpha, state_list1, shift_list1)


100%|█| 2000/2000 [03:47<00:00,  8.80it/s, Energy=-1.553904+0.000017j ± 0.000062


In [57]:
state_list2 = [gs,st1]
shift_list2 = [0.55,0.35]
eng2, state2, st2 = runDescentCS(N,B,Ak,alpha, state_list2, shift_list2)

100%|█| 800/800 [02:12<00:00,  6.03it/s, Energy=-1.12204+0.00017j ± 0.00051 [σ²=


In [5]:
overlap = np.dot(state.conj().reshape(2**N, 1).T, v[0].reshape(2**N, 1))
print(1 - (np.linalg.norm(overlap))**2)

0.0004002030441700999
